<a href="https://colab.research.google.com/github/Raymondgwangryeol/Raymondgwangryeol/blob/main/Study/ML/PytorchTutorial/PytorchTutorial_7_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **모델 매개변수 최적화하기**


---


모델과 데이터가 준비되었으니, 데이터에 매개변수를 최적화해서 모델 학습, 검증, 테스트를 드디어 한다 와...

<br>

### **[모델 학습과정]**
반복을 통해 학습!
<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;각 반복 단계에서 출력 추측<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**↓**   
&nbsp;&nbsp;&nbsp;&nbsp;추측과 정답 사이의 오류(loss) 계산<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**↓**   
매개변수에 대한 오류의 도함수(derivaticve)수집<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**↓**    
경사하강법(상황에 따라 알맞은 최적화 알고리즘 사용)사용해서 파라미터 최적화(optimize)

In [ ]:
#기본(pre-requisite) 코드
#이전 장인 Dataset과 DataLoader와 신경망 모델 구성하기에서 코드를 가져왔습니다.

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

device=(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
training_data = datasets.FashionMNIST(
    root = ".",
    train = True,
    download = True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root=".",
    train=False,
    download = True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to(device)

## **하이퍼파라미터(Hyperparameter)**

하이퍼 파라미터는 파라미터와 구분되어, 사용자가 딥러닝을 위해 설정하는 조절 가능한 값들을 모두 지칭한다. 모델 최적화 과정을 제어할 수 있고, 서로 다른 하이퍼 파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있다.

### **[학습 시 정의되는 하이퍼파라미터]**
- **epoch 수:** 데이터 셋을 반복하는 횟수
- **batch size:** 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수
- **학습률(learning rate):** 각 batch 또는 epoch에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있음.

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## **최적화 단계(Optimization Loop)**

하이퍼 파라미터를 설정한 뒤, 최적화 단계를 통해 모델을 학습하고 최적화할 수 있다. 최적화 단계의 각 반복(iteration)을 epoch라고 한다.

하나의 epoch은 2가지 단계로 나누어진다.
- **학습 단계(train loop):** 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
- **검증/테스트 단계(validation/test loop):** 모델 성능이 개선되고 있는지 확인을 위해 테스트 데이터셋 반복

학습 단계에서 알아야 하는 몇 가지 개념들을 간략히 알아보자.

<br><br>

### **손실 함수(loss function)**

학습용 데이터를 학습되지 않은 신경망에 넣었을 때, 정답인 결과를 얻기는 어렵다. 정답에 가까운 결과를 얻기 위해, **학습을 통해 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정**하는데(loss), 이를 측정하는 함수가 손실 함수이다. loss값을 최소화(0에 거의 수렴)하는 것이 목표이다.

일반적으로 쓰이는 손실함수로는 회귀 문제(regression task, 예측 결과가 연속성을 지님)에 쓰이는 **nn.MSELoss**(평균 제곱 오차(MSE; Mean Square Error))나 분류(classification)에 사용하는 **nn.NLLLoss**(Negative Log Likelihood), 그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 **nn.CrossEntropyLoss**등이 있다.


In [ ]:
#모델의 출력 logit을 nn.CrossEntropyLoss에 전달하여,
#logit을 정규화하고 예측 오류를 계산합니다.
loss_fn = nn.CrossEntropyLoss()

### **옵티마이저(Optimizer)**
최적화란, 각 학습 단계에서 loss를 줄이기 위해 모델의 매개변수를 조정하는 과정을 말한다. 최적화 알고리즘은 이 과정이 수행되는 방식을 정의한다.

모든 최적화 절차(logic)은 optimizer객체에 캡슐화(encapsulate)되며, PyTorch에서는 ADAM이나 RMSProp과 같은 다양한 옵티마이저를 제공하고 있다.

In [ ]:
#확률적 경사하강법(SGD, Stochastic Gradient Descent)를 최적화 함수로 사용.
#학습하려는 모델의 매개변수와 학습률을 등록하여 옵티마이저를 초기화 합니다.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계(loop)에서 최적화는 3단계로 이뤄진다.
1. ***optimizer.zero_grad()* 호출해서 모델 매개변수의 gradient 값 재설정.**
    >변화도는 기본적으로 더해지기(add up)때문에 중복 계산을 막기 위해 반복할 때 마다 0으로 initialize 함.
2. ***loss.bacwards()*로 예측 손실(prediction loss) 역전파.**
    >PyTorch는 각 매개변수에 대한 손실 변화도를 저장함.

3. **Gradient 계산 뒤 *optimizer.step()*을 호출, 역전파 단계에서 수집된 기울기값으로 매개변수 조정**

## **전체 구현**

아래는 최적화 코드를 반복하여 수행하는 train_loop와 테스트 데이터로 모델의 성능을 측정하는 test_loop를 정의한 코드이다.

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        # 예측(ptrediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct =0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실 함수와 옵티마이저를 초기화하고, train_loop와 test_loop에 전달한다. 모델의 성능 향상을 알아보기 위해, 자유롭게 epoch수를 증가시킬 수 있다.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
----------------------
loss: 2.302871  [   64/60000]
loss: 2.287245  [ 6464/60000]
loss: 2.261708  [12864/60000]
loss: 2.251158  [19264/60000]
loss: 2.235386  [25664/60000]
loss: 2.199327  [32064/60000]
loss: 2.203393  [38464/60000]
loss: 2.169182  [44864/60000]
loss: 2.164755  [51264/60000]
loss: 2.119562  [57664/60000]
Test Error: 
 Accuracy: 50.2%, Avg loss: 2.124042 

Epoch 2
----------------------
loss: 2.137970  [   64/60000]
loss: 2.120782  [ 6464/60000]
loss: 2.054124  [12864/60000]
loss: 2.072233  [19264/60000]
loss: 2.016366  [25664/60000]
loss: 1.950552  [32064/60000]
loss: 1.970934  [38464/60000]
loss: 1.887004  [44864/60000]
loss: 1.895324  [51264/60000]
loss: 1.808978  [57664/60000]
Test Error: 
 Accuracy: 60.3%, Avg loss: 1.818489 

Epoch 3
----------------------
loss: 1.853732  [   64/60000]
loss: 1.815678  [ 6464/60000]
loss: 1.695454  [12864/60000]
loss: 1.743981  [19264/60000]
loss: 1.637595  [25664/60000]
loss: 1.593030  [32064/60000]
loss: 1.603603  [38464/